In [1]:
# Mitigate all is a high level script that runs all the mitigation techniques.
# It can be found in Tools/bin/recipes/mitigate_all.sh

# It performs a number of steps including:
# 1. Double Fault mitigation
# 2. Kernel Text Fault mitigation
# 3. Kernel Debug Trap mitigation

# The first two make application code able to run in supervisor mode.
# The third makes it possible to pull data out of the kernel at known addresses.

# We detail the individual mitigations in the other notebooks in this directory
# But provide an overview here. In particular, we demonstrate how the
# Interrupt Descriptor Table (IDT) is modified by the mitigations

In [2]:
# This cell is well explained in idt.ipynb, so we skip it.
import ctypes
import os
import subprocess
import pandas as pd


IDT_ENTRIES=256

symbios_path = "../../"
os.environ["LD_LIBRARY_PATH"] += ":" + symbios_path + "Symlib/dynam_build"

def run_cmd(cmd):
    ret = subprocess.run(cmd.split())
    if ret.returncode != 0:
        print("error: ", ret.returncode)

def extract_desc_value(output, value):
    ret = output.decode("utf-8").split(value)[1].split("\n")[0]
    return hex(int(ret, 16))

def get_all_desc_values(df, output):
    fields = ["full addr:", "segment:", "ist:", "zero0:", "type:", "dpl:", "p:"]
    values = []
    for field in fields:
        value = extract_desc_value(output, field)
        values.append(value)
    df.loc[len(df)] = values

def read_whole_idt():
    import pandas as pd
    df = pd.DataFrame(columns=["full addr:", "segment:", "ist:", "zero0:", "type:", "dpl:", "p:"])
    for i in range(IDT_ENTRIES):
        output = subprocess.check_output([symbios_path + "Tools/bin/idt_tool", "-p", "-v", str(i)])
        get_all_desc_values(df, output)
    return df

In [3]:
# Here we read the whole IDT into a dataframe before and after the mitigations.

old_idt = read_whole_idt()

run_cmd(symbios_path + "Tools/bin/recipes/mitigate_all.sh")

new_idt = read_whole_idt()


ffffc90001a5b000
ffffc90001a77000
ffffc90001a8f000
mitigated core 0
mitigation finished


In [12]:
# We see that len(modified_entries) = 3, and the indices correspond to the 
# double falut, text fault, and debug trap mitigations.

# Here we check which entries of the IDT have changed.
modified_entries = [i for i in range(IDT_ENTRIES) if not old_idt.iloc[i].equals(new_idt.iloc[i])]
print("Modified entries: ", len(modified_entries), modified_entries)

Modified entries:  3 [1, 8, 14]


In [13]:
# We can dig in further here, but we leave the discussion to the individual
# mitigation notebooks.
for i in modified_entries:
    print("Entry ", i)
    print(old_idt.iloc[i])
    print(new_idt.iloc[i])

Entry  1
full addr:    0xffffc90001a4f000
segment:                    0x10
ist:                         0x3
zero0:                       0x0
type:                        0xe
dpl:                         0x0
p:                           0x1
Name: 1, dtype: object
full addr:    0xffffc90001a9f000
segment:                    0x10
ist:                         0x3
zero0:                       0x0
type:                        0xe
dpl:                         0x0
p:                           0x1
Name: 1, dtype: object
Entry  8
full addr:    0xffffc90000593000
segment:                    0x10
ist:                         0x1
zero0:                       0x0
type:                        0xe
dpl:                         0x0
p:                           0x1
Name: 8, dtype: object
full addr:    0xffffc90001a67000
segment:                    0x10
ist:                         0x1
zero0:                       0x0
type:                        0xe
dpl:                         0x0
p:                    